[View in Colaboratory](https://colab.research.google.com/github/gowtham91m/Analytics_Vidhya_hackathon/blob/master/enigma.ipynb)

In [0]:
%%capture
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge,LassoCV, Lasso
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures
from google.colab import files
from sklearn.cross_validation import *
from time import time

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
train = drive.CreateFile({'id': '10ihNrtkuPvwlVEODXmw9N2etRWBxTdmh'})
test = drive.CreateFile({'id': '10XhWYDZZYiW0jx8tzj0iijKiEFwUXTrk'})

train.GetContentFile('train.csv')
test.GetContentFile('test.csv')

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_id = train.ID
test_id = test.ID
data = train.append(test)

dummies = pd.get_dummies(data.Tag,drop_first=True)
data = pd.concat([data,dummies],axis=1)
#data.Answers=np.log(data.Answers+1)
#data.Reputation=np.log(data.Reputation+1)
# drop username
data.drop(['ID','Username','Tag'],axis=1,inplace=True)

train = data.loc[~data.Upvotes.isnull()]
test=data.loc[data.Upvotes.isnull()]
test = test.drop(['Upvotes'],axis=1)

y = train.Upvotes
X = train.drop(['Upvotes'],axis=1)

#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
#dtrain = xgb.DMatrix(X_train, label=y_train)
#dtest = xgb.DMatrix(X_test, label=y_test)
#dtrain = xgb.DMatrix(X,y)
#test_matrix = xgb.DMatrix(test)
#Std = StandardScaler().fit(X)
#X = Std.transform(X)
#X_test = Std.transform(test)


poly = PolynomialFeatures(degree=2)
x_poly = poly.fit_transform(X)
test_poly = poly.fit_transform(test)

In [0]:
def submit_file(model):
  pred = model.predict(test_poly3)
  submit = pd.DataFrame({'ID':test_id,'Upvotes':pred})
  submit.to_csv('submit.csv',index=False)
  files.download('submit.csv')

## grid search cv
def rmse(y_true,y_pred):
  return np.sqrt(mean_squared_error(y_true,y_pred))
rmse_score = make_scorer(rmse,greater_is_better=False)

def gridsearchcv(xtrain,ytrain,model,params, scoring = None, fit_params = None):
  grid = GridSearchCV(model
                      ,params
                      ,scoring = scoring
                      ,fit_params = fit_params
                     # ,cv=StratifiedKFold(ytrain, n_folds=2,  shuffle=True)
                      ,cv = 4
                      ,verbose = 1
                      ,refit=True)
  grid.fit(xtrain,ytrain)
  
  best_parameters, score, _ = max(grid.grid_scores_, key=lambda x: x[1])
  print('Score:', np.sqrt(-score))
  for param_name in sorted(best_parameters.keys()):
      print("%s: %r" % (param_name, best_parameters[param_name]))
  return grid

def randomizedcv(xtrain,ytrain,model,params, scoring = None, fit_params = None):
  grid = RandomizedSearchCV(model, params, 
                   n_jobs=5, 
                   cv=3, 
                   scoring=scoring,
                   verbose=1,
                   refit=True)
  grid.fit(xtrain,ytrain)
  
  best_parameters, score, _ = max(grid.grid_scores_, key=lambda x: x[1])
  print('score:', np.sqrt(-score))
  for param_name in sorted(best_parameters.keys()):
      print("%s: %r" % (param_name, best_parameters[param_name]))
  return grid

In [0]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR, NuSVR, LinearSVR
svr_poly = NuSVR().fit(x_poly,y)
submit_file(svr_poly)

In [0]:
poly3 = PolynomialFeatures(degree=3)
x_poly3 = poly3.fit_transform(X)
test_poly3 = poly3.fit_transform(test)


In [0]:
t = time()
lr_poly3 = LinearRegression().fit(x_poly3,y)
print(time() - t)

In [0]:
submit_file(lr_poly3)

In [0]:
#submit_file(model)

In [0]:
# solve no space left on the device
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [0]:
# xbg
params = {'min_child_weight':[4,5], 
          'gamma':[i/10.0 for i in range(3,6)],  
          'subsample':[i/10.0 for i in range(6,11)],
          'colsample_bytree':[i/10.0 for i in range(6,11)],
          'max_depth': [2,3,4]}

xgb = XGBRegressor()
model = randomizedcv(X_poly,y,xgb,params,'neg_mean_squared_error')

Fitting 2 folds for each of 10 candidates, totalling 20 fits


[Parallel(n_jobs=5)]: Done  20 out of  20 | elapsed: 13.3min finished


score: 1231.1026226384338
colsample_bytree: 0.9
gamma: 0.4
max_depth: 3
min_child_weight: 4
subsample: 0.7


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [0]:
#kernel=’rbf’, degree=3, gamma=’auto’, coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1
from sklearn.svm import SVR, NuSVR, LinearSVR
svr_poly = NuSVR().fit(x_poly,y)
submit_file(svr_poly)

In [0]:
from sklearn.linear_model import OrthogonalMatchingPursuit
print(cross_val_score(OrthogonalMatchingPursuit(),X,y,cv=5,scoring=rmse_score))

[-4014.68638108 -3178.64437153 -2716.14799671 -2657.1726923
 -3407.39592938]


In [0]:
# ridge
ridge_params = {'alpha':[0.5,1.0,1.5] }
ridge_model = gridsearchcv(x_poly,y,Ridge(),ridge_params,'neg_mean_squared_error')

Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   11.2s finished


Score: 1196.4622603976597
alpha: 1.5


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [0]:
# linear regression
from sklearn.linear_model import SGDRegressor,BayesianRidge
#print(cross_val_score(LinearRegression(),x_poly,y,cv=5,scoring=rmse_score))
print(cross_val_score(BayesianRidge(),x_poly,y,cv=5,scoring=rmse_score))
print(cross_val_score(SGDRegressor(),X,y,cv=5,scoring=rmse_score))

[-1242.16512203 -1551.95177437  -924.64766833  -866.45532714
 -1344.9176761 ]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/dist-packages/skl

[-6.31385956e+18 -1.54664559e+18 -7.60638178e+18 -1.29512456e+18
 -2.37914354e+19]


In [0]:
from sklearn.preprocessing import PolynomialFeatures
print(cross_val_score(LinearRegression(),X_poly,y,cv=5,scoring=rmse_score))

[-1242.91420687 -1550.94930774  -925.46876226  -866.76504859
 -1345.87700275]


In [0]:

poly = PolynomialFeatures(degree=2)
x_poly = poly.fit_transform(X)
test_poly = poly.fit_transform(test)

In [0]:
lr_poly = LinearRegression().fit(X_poly,y)
submit_file(lr_poly)

In [0]:
lr_poly = SGDRegressor().fit(X_poly,y)
submit_file(lr_poly)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
